In [16]:
import pandas as pd
import numpy as np
from random import sample
import re,os,pickle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import gensim.downloader as api
from gensim.models.keyedvectors import Word2VecKeyedVectors
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from scipy import spatial
from nltk.corpus import stopwords

In [17]:
# model = api.load('glove-twitter-25')
# pickle.dump(model,open('glove-twitter-25.pkl','wb'))
model2=pickle.load(open('glove-twitter-25.pkl','rb'))
# model2 = api.load('word2vec-google-news-300')

In [18]:
def data_prepare(file_path="/home/shiva/Desktop/datasets/"):
    f1=os.path.join(file_path,"chatting1.csv")
    f2=os.path.join(file_path,"chatting2.csv")
    f3=os.path.join(file_path,"chatting3.csv")
    f4=os.path.join(file_path,"fake_data.csv")
    f5=os.path.join(file_path,"GeneralNewsArticle.csv")
    f6=os.path.join(file_path,"fake_message_dataset.csv")
    f7=os.path.join(file_path,"vishwas.csv")
    df1=pd.read_csv(f1)
    df2=pd.read_csv(f2)
    df3=pd.read_csv(f3)
    df4=pd.read_csv(f4)
    df5=pd.read_csv(f5,nrows=7000)
    df6=pd.read_csv(f6)
    df7=pd.read_csv(f7)
#     Preparing df1 
    print(" dataset  length : ",len(df1))
    df1['bin_cat']=df1.category.map({'chatting':0})
    df1.dropna(inplace=True)
    print(df1.describe())
    print(df1.head())
    # Preparing df2 
    print(" dataset 2 length : ",len(df2))
    df2['bin_cat']=df2.category.map({'chatting':0})
    df2.dropna(inplace=True)
    print(df2.describe())
    print(df2.head())
    # Preparing df3 
    print(" dataset 3 length : ",len(df3))
    df3['bin_cat']=df3.category.map({'chatting':0})
    df3.rename(columns={'txt':'text'},inplace=True)
    df3.dropna(inplace=True)
    print(df3.describe())
    print(df3.head())
    # Preparing df4 
    print(" dataset 4 length : ",len(df4))
    df4['bin_cat']=df4.category.map({'fake':1})
    df4.dropna(inplace=True)
    print(df4.describe())
    print(df4.head())
    # Preparing df5 
    print(" dataset 5 length : ",len(df5))
    df5['bin_cat']=df5.category.map({'General News':0})
    df5.dropna(inplace=True)
    print(df5.describe())
    print(df5.head())
    # Preparing df6 
    print(" dataset 6 length : ",len(df6))
    df6['bin_cat']=df6.category.map({'fake':1})
    df6.dropna(inplace=True)
    print(df6.describe())
    print(df6.head())
    # Preparing df7 
    print(" dataset 7 length : ",len(df7))
    df7['bin_cat']=df7.category.map({'fake':1})
    df7.dropna(inplace=True)
    print(df7.describe())
    print(df7.head())
    # Organizing datasets
    c1,c2,c3='title','text','bin_cat'
    objs=[df1[[c2,c3]],df2[[c2,c3]],df3[[c2,c3]],df4[[c2,c3]],df5[[c2,c3]],df6[[c2,c3]],df7[[c2,c3]]]
    df=pd.concat(objs)
    df.dropna(inplace=True)
    print("Whole dataset length : ",len(df))
    return df
df=data_prepare()
df['category']=df['bin_cat']
df.dropna(inplace=True)


def get_only_chars(line):

    clean_line = ""

    line = line.replace("’", "")
    line = line.replace("'", "")
    line = line.replace("-", " ") #replace hyphens with spaces
    line = line.replace("\t", " ")
    line = line.replace("\n", " ")
    line = line.lower()

    for char in line:
        if char in 'qwertyuiopasdfghjklzxcvbnm':
            clean_line += char
        else:
            clean_line += ' '

    clean_line = re.sub(' +',' ',clean_line) #delete extra spaces
    return clean_line

# Text processing (split, find token id, get embedidng)
def transform_sentence(text, model):

    """
    Mean embedding vector
    """

    def preprocess_text(raw_text, model=model):

        """ 
        Excluding unknown words and get corresponding token
        """

        raw_text = raw_text.split()

        return list(filter(lambda x: x in model.vocab, raw_text))

    tokens = preprocess_text(text)

    if not tokens:
        return np.zeros(model.vector_size)

    text_vector = np.mean(model[tokens], axis=1)

    return np.array(text_vector)

 dataset  length :  1345
        Unnamed: 0  bin_cat
count  1345.000000   1345.0
mean    672.000000      0.0
std     388.412367      0.0
min       0.000000      0.0
25%     336.000000      0.0
50%     672.000000      0.0
75%    1008.000000      0.0
max    1344.000000      0.0
   Unnamed: 0 place                                               text  \
0           0  None                                                Hya   
1           1  None   Prasid ager group ta r admin R keu na.. so ok...   
2           2  None   Hmm Kalo Dal Nei Ki levelr stalk korish 😓 Ke ...   
3           3  None   Ok Bnara bol Sudeshnar sathe ki korechis Ki k...   
4           4  None   Baaltai Dekhabi na to ? Bujhe gechi Mondarmon...   

   category  bin_cat  
0  chatting        0  
1  chatting        0  
2  chatting        0  
3  chatting        0  
4  chatting        0  
 dataset 2 length :  77
       Unnamed: 0  bin_cat
count   77.000000     77.0
mean    38.000000      0.0
std     22.371857      0.0
min     

In [19]:
df['text'] = df['text'].apply(lambda x: get_only_chars(x))
X_train,X_test,y_train,y_test=train_test_split(df['text'],df['bin_cat'].values,test_size=0.2,random_state=500)

In [5]:
# from sklearn.ensemble import RandomForestClassifier
# from imblearn.under_sampling import RandomUnderSampler
# from sklearn.feature_extraction.text import TfidfVectorizer
# from imblearn.pipeline import make_pipeline as make_pipeline_imb
# from imblearn.metrics import  classification_report_imbalanced 

# def train(X_train, y_train):
# #     clf = KNeighborsClassifier(n_neighbors=1,p=2,algorithm='kd_tree')
#     clf=RandomForestClassifier(min_samples_leaf=1, min_samples_split=6, n_estimators=200, 
#                              criterion='gini', bootstrap='False', n_jobs= -1)
#     pipe = make_pipeline_imb(TfidfVectorizer(ngram_range=(1,2),norm='l2',max_features=1000),
#                          RandomUnderSampler(random_state=440),
#                          clf)
#     pipe.fit(X_train, y_train)
#     return pipe


# def test(pipe,X_test,y_test):
#     y_pred = pipe.predict(X_test)
#     probablity=pipe.predict_proba(X_test)
#     print("Random forest Accuracy:", accuracy_score(y_test, y_pred))
#     print(classification_report_imbalanced(y_test, y_pred))
#     return y_pred,probablity
# model=train(X_train, y_train)
# pred,prob=test(model,X_test,y_test)
# for i,j in zip(pred,y_test.values):
#     print(i,j)

# Oneshot Learning 

In [20]:
import random
import numpy as np
import keras,pickle
import pandas as pd
import wandb
from wandb.keras import WandbCallback
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Concatenate, Dot, Lambda, Input,Dropout,BatchNormalization,Conv2D,MaxPooling2D,LSTM,RNN,Embedding
from keras.datasets import mnist
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import OneHotEncoder

In [21]:
xls=pd.ExcelFile('Law_Wise_Text.xlsx')
df1=pd.read_excel(xls,'Copyright Law',usecols=['Copyright Law Text'])
df1['label']=['Copyright' for _ in range(len(df1))]
df1.rename(columns={'Copyright Law Text':'text'},inplace=True)
df2=pd.read_excel(xls,'Criminal Law',usecols=['Criminal Law Text'])
df2['label']=['Criminal' for _ in range(len(df2))]
df2.rename(columns={'Criminal Law Text':'text'},inplace=True)
df3=pd.read_excel(xls,'Drug Biotech Law',usecols=['Drug/Biotech Law'])
df3['label']=['DrugBiotech' for _ in range(len(df3))]
df3.rename(columns={'Drug/Biotech Law':'text'},inplace=True)
df4=pd.read_excel(xls,'Patent Law',usecols=['Patent Law Text'])
df4['label']=['Patent' for _ in range(len(df4))]
df4.rename(columns={'Patent Law Text':'text'},inplace=True)
df5=pd.read_excel(xls,'Employment Law',usecols=['Employement Law Text'])
df5['label']=['Employment' for _ in range(len(df5))]
df5.rename(columns={'Employement Law Text':'text'},inplace=True)
df=pd.concat([df1,df2,df3,df4,df5],axis=0)
# np.array().concat()
df.head()

,text,label
0,division bluesman music appellees bauer menkes...,Copyright
1,peter rovner ilana environment age pro agostin...,Copyright
2,li gould division judge product produce san ba...,Copyright
3,linda dallas month russell center appellees pa...,Copyright
4,elmore denying federal norgle similarly plaint...,Copyright


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119 entries, 0 to 23
Data columns (total 2 columns):
text     119 non-null object
label    119 non-null object
dtypes: object(2)
memory usage: 2.8+ KB


In [23]:
model2=pickle.load(open('glove-twitter-25.pkl','rb'))
print(df.nunique())
print(df.drop_duplicates('label'))
df['bin_cat']=df.label.map({'Copyright':0,'Criminal':1,'DrugBiotech':2,'Patent':3,'Employment':4})
df.head()

text     116
label      5
dtype: int64
                                                text        label
0  division bluesman music appellees bauer menkes...    Copyright
0  identifies month frivolous pro anders diane in...     Criminal
0  possession linda month amend career pro amendm...  DrugBiotech
0  carol russell fed pro civ patent diane perjuri...       Patent
0  general intellectual astrue rovner ilana baltz...   Employment


,text,label,bin_cat
0,division bluesman music appellees bauer menkes...,Copyright,0
1,peter rovner ilana environment age pro agostin...,Copyright,0
2,li gould division judge product produce san ba...,Copyright,0
3,linda dallas month russell center appellees pa...,Copyright,0
4,elmore denying federal norgle similarly plaint...,Copyright,0


In [24]:
#Tfidvectorizer 
# def tf(x):
#     tfid=TfidfVectorizer(ngram_range=(1,2),norm='l1',max_features=784)
#     return np.resize(tfid.fit_transform([x]).toarray(),(28,28))

In [25]:
df=pd.concat([df[df.bin_cat==0],df[df.bin_cat==1]],axis=0)
df['text'] = df['text'].apply(lambda x: get_only_chars(x))
X_train,X_test,y_train,y_test=train_test_split(df['text'].values,df['bin_cat'].values,test_size=0.2,random_state=420)
df.head()

,text,label,bin_cat
0,division bluesman music appellees bauer menkes...,Copyright,0
1,peter rovner ilana environment age pro agostin...,Copyright,0
2,li gould division judge product produce san ba...,Copyright,0
3,linda dallas month russell center appellees pa...,Copyright,0
4,elmore denying federal norgle similarly plaint...,Copyright,0


In [12]:
df.tail()

,text,label,bin_cat
19,standing set inferred federal wife witness sen...,Criminal,1
20,amendment armour leonard appealable round one ...,Criminal,1
21,even drive november month hardy holderman lake...,Criminal,1
22,kerr hamilton craig general judge lover incorp...,Criminal,1
23,part briley tying sachs peremptories joshua fi...,Criminal,1


In [13]:
# # vectorizing 
# X_train_mean = X_train.apply(lambda x : transform_sentence(x, model2))
# X_test_mean = X_test.apply(lambda x : transform_sentence(x, model2))
# X_train_mean = pd.DataFrame(X_train_mean)['text'].apply(pd.Series)
# X_test_mean = pd.DataFrame(X_test_mean)['text'].apply(pd.Series)

# X_train_mean_=X_train_mean.iloc[:,:100]
# X_train_mean_=pd.concat([X_train_mean_,y_train],axis=1)
# X_train_mean_['label']=y_train.values
# X_train_mean_=X_train_mean_.dropna()
# y_train=X_train_mean_.label
# X_train_mean=X_train_mean_.iloc[:,:100]

# X_test_mean_=X_test_mean.iloc[:,:100]
# X_test_mean_=pd.concat([X_test_mean_,y_test],axis=1)
# X_test_mean_['label']=y_test
# X_test_mean_=X_test_mean_.dropna()
# y_test=X_test_mean_.label
# X_test_mean=X_test_mean_.iloc[:,:100]

In [14]:
from keras import backend as K
# from sklearn.decomposition import PCA

def exponent_neg_manhattan_distance(vects):
    x, y = vects
    arms_difference=x-y
    """ Compute the exponent of the opposite of the L1 norm of a vector, to get the left/right inputs
    similarity from the inputs differences. This function is used to turned the unbounded
    L1 distance to a similarity measure between 0 and 1"""
    return K.exp(-K.sum(K.abs(arms_difference), axis=1, keepdims=True))


def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

def cosine_similarity(vects):
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    x, y = vects
    return K.sum(x*y,axis=1, keepdims=True)/K.sqrt(K.sum(x*x,axis=1, keepdims=True)*K.sum(y*y,axis=1, 
                                                                                          keepdims=True))

In [58]:

# def get_siamese_model(input_shape):
#     """
#         Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
#     """
#     left_input = Input(input_shape)
#     right_input = Input(input_shape)
#     model = Sequential()
#     model.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape))
#     model.add(MaxPooling2D())
#     model.add(Conv2D(128, (7,7), activation='relu'))
#     model.add(MaxPooling2D())
#     model.add(Conv2D(128, (4,4), activation='relu'))
#     model.add(MaxPooling2D())
#     model.add(Conv2D(256, (4,4), activation='relu'))
#     model.add(Flatten())
#     model.add(Dense(4096, activation='sigmoid'))
#     encoded_l = model(left_input)
#     encoded_r = model(right_input)
#     L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
#     L1_distance = L1_layer([encoded_l, encoded_r])
#     prediction = Dense(1,activation='sigmoid')(L1_distance)
#     siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
#     return siamese_net
# mod = get_siamese_model((105, 105, 1))
# mod.summary()

# LSTM

In [26]:
# Doc2vec
import gensim
from gensim.models import Doc2Vec
def doc2vec(X_train):
    document_tagged = []
    tagged_count = 0
    for line in X_train:
        document_tagged.append(gensim.models.doc2vec.TaggedDocument(line,[tagged_count]))
        tagged_count +=1 
    d2v = Doc2Vec(document_tagged)
    d2v.train(document_tagged,epochs=d2v.epochs,total_examples=d2v.corpus_count)
    return d2v.docvecs.vectors_docs

print(X_train.shape,y_train.shape)
X_train=doc2vec(X_train)
X_test=doc2vec(X_test)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)
### GENERATE PAIR SAMPLES FOR SIAMESE NETWORK ###

def pairs(X_train,Y_train):
    sequence_length=10
    left_input = []
    right_input = []
    targets = []
    pairs = 2
    features=['text']
    for i in range(len(Y_train)):
        for _ in range(pairs):
            compare_to = i
            while compare_to == i:
                compare_to = random.randint(0,len(Y_train)-1)
            left_input.append(X_train[i])
            right_input.append(X_train[compare_to])
            if Y_train[i] == Y_train[compare_to]: # They are the same
                targets.append(1.)
            else:# Not the same
                targets.append(0.)

    left_input = np.asarray(left_input).reshape(-1, sequence_length, 10)# len(features)
    right_input = np.asarray(right_input).reshape(-1, sequence_length, 10)# len(features)
    targets = np.asarray(targets)
    print(left_input.shape, right_input.shape, targets.shape)
    return left_input,right_input,targets
left_input,right_input,targets=pairs(X_train,y_train)
left_input_,right_input_,targets_=pairs(X_test,y_test)

### DEFINE SIAMESE NETWORK ARCHITECTURE ###

test_size = 0.1
def SiamesNet(shape=(10,10)):
    
    left_input = Input(shape)
    right_input = Input(shape)

    lstmnet = Sequential([
        BatchNormalization(),
        LSTM(128, activation='relu', return_sequences=True, 
             input_shape=shape),
        LSTM(32, activation='relu')
    ])

    encoded_l = lstmnet(left_input)
    encoded_r = lstmnet(right_input)

    L1_layer = Lambda(lambda tensor: K.abs(tensor[0] - tensor[1]))

    L1_distance = L1_layer([encoded_l, encoded_r])
    drop = Dropout(0.2)(L1_distance)
    prediction = Dense(1,activation='sigmoid')(drop)
    model = Model(inputs=[left_input,right_input],outputs=prediction)
    
    return model
### FIT SIAMESE NETWORK ###

siamese_net = SiamesNet()
siamese_net.compile(loss="mean_squared_error", optimizer='adam', metrics=['accuracy'])

siamese_net.fit([left_input, right_input], targets, batch_size=32, epochs=50, 
                validation_split=0.1, verbose=2)

(38,) (38,)
(38, 100) (38,) (10, 100) (10,)
(76, 10, 10) (76, 10, 10) (76,)
(20, 10, 10) (20, 10, 10) (20,)
Train on 68 samples, validate on 8 samples
Epoch 1/50
 - 5s - loss: 0.2499 - accuracy: 0.5147 - val_loss: 0.2500 - val_accuracy: 0.5000
Epoch 2/50
 - 0s - loss: 0.2491 - accuracy: 0.5882 - val_loss: 0.2501 - val_accuracy: 0.5000
Epoch 3/50
 - 0s - loss: 0.2482 - accuracy: 0.5735 - val_loss: 0.2501 - val_accuracy: 0.5000
Epoch 4/50
 - 0s - loss: 0.2476 - accuracy: 0.5735 - val_loss: 0.2502 - val_accuracy: 0.5000
Epoch 5/50
 - 0s - loss: 0.2460 - accuracy: 0.5735 - val_loss: 0.2502 - val_accuracy: 0.5000
Epoch 6/50
 - 0s - loss: 0.2457 - accuracy: 0.5735 - val_loss: 0.2503 - val_accuracy: 0.5000
Epoch 7/50
 - 0s - loss: 0.2442 - accuracy: 0.5735 - val_loss: 0.2504 - val_accuracy: 0.5000
Epoch 8/50
 - 0s - loss: 0.2409 - accuracy: 0.5735 - val_loss: 0.2505 - val_accuracy: 0.5000
Epoch 9/50
 - 0s - loss: 0.2436 - accuracy: 0.5735 - val_loss: 0.2506 - val_accuracy: 0.5000
Epoch 10/50


In [108]:
def check(num):
    if num>0.7:
        return 1
    else:
        return 0
n1=np.random.randint(0,1600)
n2=np.random.randint(0,1600)
print('loc of f1,f2 : ',n1,n2)
x1 = np.resize(transform_sentence(df['text'].values[n1], model2),(1,10,10))
x2 = np.resize(transform_sentence(df['text'].values[n2], model2),(1,10,10))
print(df['bin_cat'].values[n1],df['bin_cat'].values[n2])
print('probablity of verification of matching ',check(model.predict([x1,x2])[0][0]))

loc of f1,f2 :  313 1187
0 3
probablity of verification of matching  0
